In [58]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn import naive_bayes, svm, metrics
import pandas as pd
import numpy as np
# reset colwitdth options when running all cells 
pd.reset_option('display.max_colwidth')

### Load dataset (all tweets and corresponding stock prices)

In [59]:
data = pd.read_json('processed_data/data_merged.json')

In [60]:
data.head()

,timestamp,hashtags,text,username,likes,replies,retweets,Open,Close,PriceUp
0,2018-01-02,"[Tesla, ModelS]","In the past 2 years, I've driven 18,823 miles ...",Ben Sullins 💪,110,6,10,312.0,320.53,True
1,2018-01-02,"[Tesla, P90D, Blog, Youtube]",Ya estamos en @louesfera probando un #Tesla #P...,Fco Javier,2,1,2,312.0,320.53,True
2,2018-01-02,"[Snapchat, Uber, Twitter, Facebook, Tesla, Goo...",Here's how old these companies will be turning...,Imran,53,7,41,312.0,320.53,True
3,2018-01-02,[Muskwatchpic],"From SpaceX to Tesla, here are our biggest que...",Nerdist,37,5,10,312.0,320.53,True
4,2018-01-02,"[Braunschweig, VW, Tesla]",In #Braunschweig produziert #VW seine Batterie...,HAZ,5,3,2,312.0,320.53,True


## TfIdf-Vectorize the data and apply PCA to reduce vector dimensionality 
Use each tweet and predict whether it was written on a day where stock price has grown (PriceUp == True) or not

As classificators use the algorithms learned in class: naive bayes and SVMs

In [61]:
# generate the train and test sets
tweets_train, tweets_test, labels_train, labels_test = train_test_split(data['text'], data['PriceUp'], 
                                                   test_size=0.15, random_state=333, shuffle=True)
len(tweets_train)

4206

In [62]:
# vectorize train and test data with TF-IDF
vectorizer = TfidfVectorizer()
train_matrix = vectorizer.fit_transform(tweets_train)
test_matrix = vectorizer.transform(tweets_test)

print("The dimension of a tweet vector is ", train_matrix.shape[1])
print("The type of the train_matrix is ", type(train_matrix))

The dimension of a tweet vector is  22108
The type of the train_matrix is  <class 'scipy.sparse.csr.csr_matrix'>


In [63]:
# convert sparse matrix into a numpy matrix
train_matrix = train_matrix.todense()
train_matrix.shape

(4206, 22108)

### PCA
Our vector space is far higher than the number of training data. Therefore the classifier will for sure overfit and not generalize well to the test data at all. To overcome this, we apply PCA and drastically reduce the dimensionality of the tweet vectors remaining only 90% of explained variance.

In [67]:
# PCA requires a data normalization (zero mean and unit variance)
from sklearn.preprocessing import StandardScaler
normalizer = StandardScaler()
print(f"Min/Max values before normalization: {np.min(train_matrix)}, {np.max(train_matrix)}")

# Fit normalizer and transform train and test data
# normalizer has to be fitted only on training data
train_matrix = normalizer.fit_transform(train_matrix)
test_matrix = normalizer.transform(test_matrix)
print(f"Min/Max values before normalization: {np.min(train_matrix)}, {np.max(train_matrix)}")



Min/Max values before normalization: -1.4834715569020316, 64.84597092407806


ValueError: Cannot center sparse matrices: pass `with_mean=False` instead. See docstring for motivation and alternatives.

### Observations
Unfortunately, sklearn cannot normalize a big sparse matrix to have zero mean (https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html?highlight=standardscaler#sklearn.preprocessing.StandardScaler), which is mandatory to apply PCA. A simple alternative seems to be not around. Therefore we first stop further investigating this direction.

#### Please ignore the following code
Only remained in case we find an alternative way to reduce the data dimensionality.

In [64]:
means = np.mean(train_matrix, axis=0)
stds = np.std(train_matrix, axis=0)
stds.shape

(1, 22108)

In [65]:
epsilon = 1e-10
print(f"Min/Max values before normalization: {np.min(train_matrix)}, {np.max(train_matrix)}")
train_matrix = (train_matrix - means)/(stds+epsilon)
print(f"Min/Max values before normalization: {np.min(train_matrix)}, {np.max(train_matrix)}")

Min/Max values before normalization: 0.0, 1.0
Min/Max values before normalization: -1.4834715569020316, 64.84597092407806


In [8]:
svm_classifier = svm.LinearSVC(max_iter=int(1e6))
svm_classifier.fit(train_matrix, labels_train)

nb_classifier = naive_bayes.GaussianNB()
nb_classifier.fit(train_matrix.toarray(), labels_train)

GaussianNB(priors=None, var_smoothing=1e-09)

#### Test if training was successful
As we have not enough data, a working classifier should overfit to the training data and hence perfectly predict the labels of the training set.

In [9]:
# check if classifier has really overfitted to the data by testing it on the training data
preds_svm = svm_classifier.predict(train_matrix)
svm_acc = metrics.accuracy_score(labels_train, preds_svm)

preds_nb = nb_classifier.predict(train_matrix.toarray())
nb_acc = metrics.accuracy_score(labels_train, preds_nb)


svm_prec, svm_rec, svm_fscore, svm_sup = \
metrics.precision_recall_fscore_support(labels_train, preds_svm, pos_label=True, average='binary')

nb_prec, nb_rec, nb_fscore, nb_sup = \
metrics.precision_recall_fscore_support(labels_train, preds_nb, pos_label=True, average='binary')

print('   \t\tSVM \t\tNaive Bayes')
print('Acc \t\t {0:.3f} \t\t {1:.3f}'.format(svm_acc, nb_acc))
print('Prec \t\t {0:.3f} \t\t {1:.3f}'.format(svm_prec, nb_prec))
print('Rec \t\t {0:.3f} \t\t {1:.3f}'.format(svm_rec, nb_rec))
print('FMeas \t\t {0:.3f} \t\t {1:.3f}'.format(svm_fscore, nb_fscore))

   		SVM 		Naive Bayes
Acc 		 0.996 		 0.977
Prec 		 0.995 		 1.000
Rec 		 0.997 		 0.957
FMeas 		 0.996 		 0.978


Check! Both classifiers reach an almost 100% accuracy on the training data. Therefore we can be sure, the classifier really learned a model based on the training data.

### Test learned models on the test set

In [10]:
# test the classifiers
preds_svm = svm_classifier.predict(test_matrix)
svm_acc = metrics.accuracy_score(labels_test, preds_svm)

preds_nb = nb_classifier.predict(test_matrix.toarray())
nb_acc = metrics.accuracy_score(labels_test, preds_nb)


svm_prec, svm_rec, svm_fscore, svm_sup = \
metrics.precision_recall_fscore_support(labels_test, preds_svm, pos_label=True, average='binary')

nb_prec, nb_rec, nb_fscore, nb_sup = \
metrics.precision_recall_fscore_support(labels_test, preds_nb, pos_label=True, average='binary')

print('   \t\tSVM \t\tNaive Bayes')
print('Acc \t\t {0:.3f} \t\t {1:.3f}'.format(svm_acc, nb_acc))
print('Prec \t\t {0:.3f} \t\t {1:.3f}'.format(svm_prec, nb_prec))
print('Rec \t\t {0:.3f} \t\t {1:.3f}'.format(svm_rec, nb_rec))
print('FMeas \t\t {0:.3f} \t\t {1:.3f}'.format(svm_fscore, nb_fscore))

   		SVM 		Naive Bayes
Acc 		 0.569 		 0.556
Prec 		 0.584 		 0.592
Rec 		 0.599 		 0.473
FMeas 		 0.592 		 0.526


To make a statement about the results, we first have to look at the distribution of labels in the test dataset.
An even simpler baseline we can use to compare our results with is a classifier that constantly predicts the class that is most common in the test set. 

In [11]:
num_trues, num_falses = labels_test.value_counts()
print("A classifier that always predicts 'True' would get an accuracy of: %.3f" % (num_trues/labels_test.count()))

A classifier that always predicts 'True' would get an accuracy of: 0.521


On the first sight, our classifier seems to have learned a very little bit, having an accuracy of 55.4 and 54.9 percent while the constant prediction would lead to 53.7 percent. In our case of predicting whether the stock price will close higher that it has opened based on a tweet, recall is much more important to us.

Altogether the difference is too insignificant and is expected to be not reproducible when using different hyper parameters like the size of the training set, another random seed etc.

**Update**: After playing a bit with hyperparameters, we can say that the accuracy of our classifiers is always slightly above the constant value predictor. The SVM classifier always reaches a higher accuracy than the NB classifier as well as a higher Recall, which is especially important for our goal as we want to avoid false negatives.

**Discussion:** It looks like this very simple classifier already has learned some patterns in the data, which is unexpected, but can be explained as follows: 

- test and training data are expected to be correlated, as the test data contains tweets from days which we've already trained on. A better evaluation: use data of new days
- Think about it again: As we're predicting only if a tweet was written on a good day for TSLA and do not consider the time a tweet was written at, it is likely to happen, that people write about the positive development of the stock price...
